# Beta Weighting for Portfolio

Beta weighting is a technique used in portfolio optimization to adjust the weights of individual securities in a portfolio based on their correlation with a benchmark index. Beta is a measure of an asset's volatility relative to the market as a whole, with a beta of 1 indicating that the asset's price moves in line with the market, while a beta greater than 1 indicates greater volatility and a beta less than 1 indicates lower volatility.

Beta weighting can help investors to better understand the risk and return characteristics of their portfolios, as well as provide a benchmark for evaluating the performance of their investments.

In [1]:
#!pip install yfinance --force-reinstall numpy==1.23.0

In [3]:
import numpy as np
import pandas as pd
from scipy import stats


from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [4]:
start = dt.datetime(2021, 1, 1)
end = dt.datetime.now()
stocks = ['^FCHI'] + ['GLE.PA', 'BNP.PA', 'CS.PA', 'ACA.PA', 'WLN.PA']
data = pdr.get_data_yahoo(stocks, start, end)
df = data.copy()
df.loc[:, 'Close']

[*********************100%%**********************]  6 of 6 completed


Ticker,ACA.PA,BNP.PA,CS.PA,GLE.PA,WLN.PA,^FCHI
Date,,,,,,
2021-01-04,10.220,43.009998,19.438000,16.702000,80.440002,5588.959961
2021-01-05,10.255,42.919998,19.430000,16.808001,80.059998,5564.600098
2021-01-06,10.670,45.290001,20.260000,17.955999,79.500000,5630.600098
2021-01-07,10.750,46.310001,20.385000,18.142000,77.139999,5669.850098
2021-01-08,10.670,45.285000,20.365000,17.778000,77.860001,5706.879883
...,...,...,...,...,...,...
2024-05-10,15.450,71.199997,33.720001,25.920000,11.630000,8219.139648
2024-05-13,15.620,71.629997,33.689999,26.260000,11.780000,8209.280273
2024-05-14,15.655,71.519997,33.419998,27.254999,12.000000,8225.799805


In [5]:
log_rtns = np.log(df.Close/df.Close.shift(1)).dropna()
log_rtns.head()

Ticker,ACA.PA,BNP.PA,CS.PA,GLE.PA,WLN.PA,^FCHI
Date,,,,,,
2021-01-05,0.003419,-0.002095,-0.000412,0.006327,-0.004735,-0.004368
2021-01-06,0.039671,0.053748,0.041830,0.066069,-0.007019,0.011791
2021-01-07,0.007470,0.022272,0.006151,0.010305,-0.030135,0.006947
2021-01-08,-0.007470,-0.022382,-0.000982,-0.020268,0.009290,0.006510
2021-01-11,-0.013683,-0.006758,-0.005663,-0.013592,-0.040094,-0.007819


## PART I: Simple Beta Calculation

Beta Calculation is done with the following formula:
> $\beta = \frac{Covariance(Market, Stock)}{Covariance(Market)}$ 

In [6]:
def beta_compute(dataframe): 
    narray = dataframe.values
    market = narray[:, -1] #Market Index is the last col
    beta = []
    for index, col in enumerate(dataframe):
        if index < len(dataframe.columns) - 1:
            #stock returns indexed by index :)
            stock = narray[:, index]
            #compute covariance
            cov = np.cov(stock, market)
            var_m = cov[1,1]
            beta.append(cov[0, 1]/var_m)
    return pd.Series(beta, dataframe.columns[:-1], name='Beta')


BetaS = beta_compute(log_rtns)
BetaS
#log_rtns.columns[:-1]

Ticker
ACA.PA    1.065203
BNP.PA    1.270888
CS.PA     0.958544
GLE.PA    1.398403
WLN.PA    1.149543
Name: Beta, dtype: float64

## PART II: Linear Regression for Beta Calculation
Linear regression must be done to get the coeff of the market and stock returns. This will give a form of $y=\beta X$ wit $X$ being a matrix with full column rank.
It is not necessary to take the Linear Regression because the Beta Estimate will have the same values (proof see at the end of Part II).

---
N.B, the Beta estimate may also be calculated with the formulas 
> $\beta^{\hat{}} = argmin||X\beta - y||_2$ or $\beta^{\hat{}} = (X^T X)^{-1} X^T y$

In [7]:
# similar to the beta_computation
def beta_regression(dataframe): 
    narray = dataframe.values
    market = narray[:, -1] 
    beta = []
    for index, col in enumerate(dataframe):
        if index < len(dataframe.columns) - 1:
            stock = narray[:, index]
            gradient = stats.linregress(market, stock)[0]
            beta.append(gradient)
    return pd.Series(beta, dataframe.columns[:-1], name='Beta')

BetaR = beta_regression(log_rtns)
BetaR

Ticker
ACA.PA    1.065203
BNP.PA    1.270888
CS.PA     0.958544
GLE.PA    1.398403
WLN.PA    1.149543
Name: Beta, dtype: float64

## Part III: Portfolio Definition & Beta Weighting


In [8]:
units = np.array([100, 250, 300, 400, 200])
CAC40prices = df.Close[-1:].values.tolist()[0]
prices = np.array([round(price, 2) for price in CAC40prices[:-1]]) # without index
#print(CAC40prices)
print(f'Prices of stocks (no index): {prices}')
values = units * prices
weight = [round(value/sum(values), 2) for value in values]
beta = round(BetaR, 2)
print(beta)
del stocks[0]
stocks

Prices of stocks (no index): [15.62 71.61 33.55 27.26 11.7 ]
Ticker
ACA.PA    1.07
BNP.PA    1.27
CS.PA     0.96
GLE.PA    1.40
WLN.PA    1.15
Name: Beta, dtype: float64


['GLE.PA', 'BNP.PA', 'CS.PA', 'ACA.PA', 'WLN.PA']

In [9]:
Portfolio = pd.DataFrame({
    'Stock': stocks,
    'Direction': 'Long',
    'Type': 'S',          # S for stock
    'Stock Price':prices,
    'Price': prices,
    'Units': units,
    'Values': values,
    'Weights': weight,
    'Beta': beta,
    'Weighted Beta': weight*beta
})
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Values,Weights,Beta,Weighted Beta
Ticker,,,,,,,,,,
ACA.PA,GLE.PA,Long,S,15.62,15.62,100,1562.0,0.04,1.07,0.0428
BNP.PA,BNP.PA,Long,S,71.61,71.61,250,17902.5,0.42,1.27,0.5334
CS.PA,CS.PA,Long,S,33.55,33.55,300,10065.0,0.24,0.96,0.2304
GLE.PA,ACA.PA,Long,S,27.26,27.26,400,10904.0,0.25,1.40,0.3500
WLN.PA,WLN.PA,Long,S,11.70,11.70,200,2340.0,0.05,1.15,0.0575


## Part IV: Adding Options to the Portfolio

In portfolio management, delta is used to measure the overall sensitivity of a portfolio to changes in the price of the underlying asset. A portfolio's delta is the sum of the deltas of all the options positions in the portfolio. A positive delta means the portfolio will benefit from an increase in the price of the underlying asset, while a negative delta means the portfolio will benefit from a decrease in the price of the underlying asset.

In [10]:
Portfolio = Portfolio.drop(['Weighted Beta', 'Weights'], axis=1)
Portfolio['Delta'] = Portfolio['Units']
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Values,Beta,Delta
Ticker,,,,,,,,,
ACA.PA,GLE.PA,Long,S,15.62,15.62,100,1562.0,1.07,100
BNP.PA,BNP.PA,Long,S,71.61,71.61,250,17902.5,1.27,250
CS.PA,CS.PA,Long,S,33.55,33.55,300,10065.0,0.96,300
GLE.PA,ACA.PA,Long,S,27.26,27.26,400,10904.0,1.40,400
WLN.PA,WLN.PA,Long,S,11.70,11.70,200,2340.0,1.15,200


In [11]:
 Options = [
     {'option':'ACA1218E10', 'underlying':'ACA', 'Price':12,
      'units': 2, 'delta': 0.627, 'direction':'Short', 'type':'C'},
     {'option':'GLE1111E33', 'underlying':'GLE', 'Price':36,
      'units': 3, 'delta': -0.425, 'direction':'Long', 'type':'P'}
 ] 

for index, row in enumerate(Options):
    Portfolio.loc[row['option']] = [row['underlying'], row['direction'], row['type'], Portfolio.loc[row['underlying']+'.PA', 'Price'], 
                                    row['Price'], row['units'], row['Price']*row['units']*100, beta[row['underlying']+'.PA'], 
                                    (row['delta']*row['units']* 100 if row['direction'] == 'Long' else -row['delta']*row['units']*100)]
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Values,Beta,Delta
Ticker,,,,,,,,,
ACA.PA,GLE.PA,Long,S,15.62,15.62,100,1562.0,1.07,100.0
BNP.PA,BNP.PA,Long,S,71.61,71.61,250,17902.5,1.27,250.0
CS.PA,CS.PA,Long,S,33.55,33.55,300,10065.0,0.96,300.0
GLE.PA,ACA.PA,Long,S,27.26,27.26,400,10904.0,1.40,400.0
WLN.PA,WLN.PA,Long,S,11.70,11.70,200,2340.0,1.15,200.0
ACA1218E10,ACA,Short,C,15.62,12.00,2,2400.0,1.07,-125.4
GLE1111E33,GLE,Long,P,27.26,36.00,3,10800.0,1.40,-127.5


## Part V: Portfolio Overview & Delta Weighting 
Weighted Delta Calculation may be done in 2 distinct ways:
* Per point, because the index is per point:
> $\delta_{weighted} = \delta * \beta * \frac{StockPrice}{IndexPrice} $
* If there is a possiblity of 1% change:
> $\delta_{\%weighted} = 0.01 * \delta * \beta * StockPrice$

It would be more understandable and representative for the client to see what happens in the 2nd case rather than per point

In [12]:
Portfolio['CAC40 Weighted Delta (1%)'] = round(Portfolio['Beta'] * Portfolio['Stock Price'] * Portfolio['Delta'] * 0.01, 2)
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Values,Beta,Delta,CAC40 Weighted Delta (1%)
Ticker,,,,,,,,,,
ACA.PA,GLE.PA,Long,S,15.62,15.62,100,1562.0,1.07,100.0,16.71
BNP.PA,BNP.PA,Long,S,71.61,71.61,250,17902.5,1.27,250.0,227.36
CS.PA,CS.PA,Long,S,33.55,33.55,300,10065.0,0.96,300.0,96.62
GLE.PA,ACA.PA,Long,S,27.26,27.26,400,10904.0,1.40,400.0,152.66
WLN.PA,WLN.PA,Long,S,11.70,11.70,200,2340.0,1.15,200.0,26.91
ACA1218E10,ACA,Short,C,15.62,12.00,2,2400.0,1.07,-125.4,-20.96
GLE1111E33,GLE,Long,P,27.26,36.00,3,10800.0,1.40,-127.5,-48.66


In [13]:
Total_profit_loss = Portfolio['CAC40 Weighted Delta (1%)'].sum()
Total_profit_loss

450.64

## Conclusion

I understood that even people working in investment bank doesn't understand what they do